In [1]:
import pandas as pd

# Первичный анализ данных

In [2]:
data_path = '../data/full_data.csv'

In [5]:
df = pd.read_csv(data_path, index_col=0)

In [6]:
df.head()

,name_1,name_2,is_duplicate
pair_id,,,
1,Iko Industries Ltd.,"Enormous Industrial Trade Pvt., Ltd.",0
2,Apcotex Industries Ltd.,Technocraft Industries (India) Ltd.,0
3,"Rishichem Distributors Pvt., Ltd.",Dsa,0
4,Powermax Rubber Factory,Co. One,0
5,Tress A/S,Longyou Industries Park Zhejiang,0
